In [1]:
from dotenv import load_dotenv
import os
# from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI

/home/hydra/Desktop/genAi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
google_api_key = os.getenv("GOOGLE_API_KEY")


In [28]:
# llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)
# Create LLM class
llm = ChatGoogleGenerativeAI(
    model= "gemini-2.0-flash",
    temperature=1.0,
    max_retries=2,
    google_api_key=google_api_key,
)

In [29]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [30]:
# creaing a tool to multiply two numbers
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    # print("hello i am a tool")
    return a - b


In [31]:
multiply.invoke({'a':2, 'b':3})

-1

In [32]:
# now binding this tool to the LLM
# llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)
# Bind tools to the model
llm_with_tool = llm.bind_tools([multiply])

In [33]:
llm_with_tool.invoke('multiply 2 and 3')

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-aac1d8f6-a971-46f8-a0e4-b4be995e4446-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': '7d2078d3-96f3-4ed4-bc13-cc3532ec65d0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 19, 'output_tokens': 5, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})

In [34]:
t = llm_with_tool.invoke("can you multiply 2 and 3?").tool_calls[0]   #her you will this will return the tool calls made by the model not actually executing them
t

{'name': 'multiply',
 'args': {'a': 2.0, 'b': 3.0},
 'id': 'f4b428ab-a1ac-4ec6-84a7-bcf0eed0bfbb',
 'type': 'tool_call'}

In [35]:
# now how to execute the tool calls
multiply.invoke(t)
# now result of this i can give to llm to get the final answer

ToolMessage(content='-1', name='multiply', tool_call_id='f4b428ab-a1ac-4ec6-84a7-bcf0eed0bfbb')

## # ----------------------now organize all 

In [36]:

query = HumanMessage('can you multiply 20 and 3?')
messages = [query]
messages

[HumanMessage(content='can you multiply 20 and 3?', additional_kwargs={}, response_metadata={})]

In [37]:
AI_mesasge = llm_with_tool.invoke(messages)  
AI_mesasge
messages.append(AI_mesasge)
messages

[HumanMessage(content='can you multiply 20 and 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 20.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-968c51bd-e71c-42d7-9c59-b7fecf157860-0', tool_calls=[{'name': 'multiply', 'args': {'a': 20.0, 'b': 3.0}, 'id': '9790fd3e-f46d-4a4d-b63b-f29319ab943c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 23, 'output_tokens': 5, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}})]

In [38]:
tool_message = multiply.invoke(AI_mesasge.tool_calls[0])  # this will execute the tool call and return the result
messages.append(tool_message)
messages

[HumanMessage(content='can you multiply 20 and 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 20.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-968c51bd-e71c-42d7-9c59-b7fecf157860-0', tool_calls=[{'name': 'multiply', 'args': {'a': 20.0, 'b': 3.0}, 'id': '9790fd3e-f46d-4a4d-b63b-f29319ab943c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 23, 'output_tokens': 5, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='17', name='multiply', tool_call_id='9790fd3e-f46d-4a4d-b63b-f29319ab943c')]

In [39]:
llm_with_tool.invoke(messages).content

'I am sorry, I made a mistake. The result of multiplying 20 and 3 is 60, not 17.'